In [1]:
import os
import joblib
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import hopsworks


In [2]:
# COMPUTE PERFORMANCE METRICS

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return {
        "rmse": np.sqrt(mse),
        "mae": mean_absolute_error(y_true, y_pred),
        "r2": r2_score(y_true, y_pred)
    }


In [3]:
# LOAD FEATURES FROM FEATURE STORE

project = hopsworks.login()
fs = project.get_feature_store()

fg = fs.get_feature_group(
    name="aqi_features",
    version=1
)

df = fg.read()
df = df.sort_values("timestamp").reset_index(drop=True)


2026-02-05 17:23:35,413 INFO: Initializing external client
2026-02-05 17:23:35,415 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-02-05 17:23:38,387 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1338518
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.70s) 


In [4]:
TARGET = "aqi"

FEATURES = [
    col for col in df.columns
    if col not in ["timestamp", TARGET, "aqi_lag_1"]
]

X = df[FEATURES]
y = df[TARGET]

In [5]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    shuffle=False
)


In [6]:
# 1ST MODEL - RIDGE REGRESSION

ridge_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0))
])

ridge_pipeline.fit(X_train, y_train)
ridge_preds = ridge_pipeline.predict(X_val)

ridge_metrics = evaluate(y_val, ridge_preds)


In [7]:
# 2ND MODEL - RANDOM FOREST

rf_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_val)

rf_metrics = evaluate(y_val, rf_preds)


In [8]:
# 3rd MODEL - NEURAL NETWORK

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

nn_model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)
])

nn_model.compile(
    optimizer="adam",
    loss="mse"
)

nn_model.fit(
    X_train_scaled,
    y_train,
    epochs=25,
    batch_size=32,
    validation_data=(X_val_scaled, y_val),
    verbose=0
)

nn_preds = nn_model.predict(X_val_scaled).flatten()
nn_metrics = evaluate(y_val, nn_preds)



2026-02-05 17:24:08,122 WARNING: From C:\Users\User\OneDrive\Desktop\AQI Predictor\venv\lib\site-packages\keras\src\backend.py:873: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


2026-02-05 17:24:08,283 WARNING: From C:\Users\User\OneDrive\Desktop\AQI Predictor\venv\lib\site-packages\keras\src\optimizers\__init__.py:309: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.


2026-02-05 17:24:08,502 WARNING: From C:\Users\User\OneDrive\Desktop\AQI Predictor\venv\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.

18/18 [==============================] - 0s 2ms/step


In [9]:
# RESULTS OF MODEL OUTPUTS 

results = pd.DataFrame.from_dict({
    "ridge": ridge_metrics,
    "random_forest": rf_metrics,
    "neural_net": nn_metrics
}, orient="index")

print(results)

                   rmse       mae        r2
ridge          0.566048  0.468709 -0.151289
random_forest  0.061978  0.009205  0.986198
neural_net     0.544079  0.432023 -0.063658


In [ ]:
# REGISTER MODELS IN HOPSWORKS MODEL REGISTRY

mr = project.get_model_registry()
os.makedirs("models", exist_ok=True)

# ======================
# REGISTER RIDGE MODEL
# ======================
joblib.dump(ridge_pipeline, "models/ridge.pkl")

ridge_reg = mr.python.create_model(
    name="aqi_predictor",
    description="Baseline Ridge Regression AQI model (no AQI lag)",
    metrics={
        "rmse": ridge_metrics["rmse"],
        "mae": ridge_metrics["mae"],
        "r2": ridge_metrics["r2"]
    }
)

ridge_reg.save("models")


# ==========================
# REGISTER RANDOM FOREST
# ==========================
joblib.dump(rf_model, "models/random_forest.pkl")

rf_reg = mr.python.create_model(
    name="aqi_predictor",
    description="Random Forest AQI predictor (best RMSE, no AQI lag)",
    metrics={
        "rmse": rf_metrics["rmse"],
        "mae": rf_metrics["mae"],
        "r2": rf_metrics["r2"]
    }
)

rf_reg.save("models")


# ==========================
# REGISTER NEURAL NETWORK
# ==========================
nn_model.save("models/neural_net")

nn_reg = mr.tensorflow.create_model(
    name="aqi_predictor",
    description="Neural Network AQI predictor (experimental, no AQI lag)",
    metrics={
        "rmse": nn_metrics["rmse"],
        "mae": nn_metrics["mae"],
        "r2": nn_metrics["r2"]
    }
)

nn_reg.save("models/neural_net")